Alex & Allan

In [3]:
import json
from uuid import uuid4
from typing import List, Dict

class Transacao:
    def __init__(self, remetente: str, destinatario: str, valor: float, motivo: str, status: str = "PENDENTE"):
        self.id = str(uuid4())
        self.remetente = remetente
        self.destinatario = destinatario
        self.valor = valor
        self.motivo = motivo
        self.status = status

class GerenciadorTransacoes:
    def __init__(self, arquivo_pendentes: str = "transacoes_pendentes.json"):
        self.arquivo_pendentes = arquivo_pendentes
        self.transacoes_pendentes: List[Transacao] = self._carregar_pendentes()

    def _carregar_pendentes(self) -> List[Transacao]:
        """Carrega transações pendentes do arquivo JSON."""
        try:
            with open(self.arquivo_pendentes, "r") as f:
                dados = json.load(f)
                return [Transacao(**t) for t in dados]
        except FileNotFoundError:
            return []

    def _salvar_pendentes(self):
        """Salva transações pendentes no arquivo JSON."""
        with open(self.arquivo_pendentes, "w") as f:
            json.dump([vars(t) for t in self.transacoes_pendentes], f, indent=4)

    def criar_transacao_aluno(self, remetente: str, destinatario: str, valor: float, motivo: str) -> Transacao:
        """
        Alunos criam transações PENDENTES (grupo → grupo).
        Salva automaticamente no JSON.
        """
        if remetente == "PROFESSOR":
            raise ValueError("Alunos não podem usar remetente 'PROFESSOR'.")

        transacao = Transacao(remetente, destinatario, valor, motivo, "PENDENTE")
        self.transacoes_pendentes.append(transacao)
        self._salvar_pendentes()
        return transacao

    def criar_transacao_professor(self, destinatario: str, valor: float, motivo: str, senha: str) -> Transacao:
        """
        Professor cria transações APROVADAS (professor → grupo).
        Autenticação básica (em produção, use JWT/OAuth).
        """
        SENHA_PROFESSOR = "senha_secreta"  # 👈 Substitua por variável de ambiente em produção

        if senha != SENHA_PROFESSOR:
            raise PermissionError("Acesso negado. Senha incorreta.")

        transacao = Transacao("PROFESSOR", destinatario, valor, motivo, "APROVADA")
        return transacao  # 👈 Não é salva em pendentes (outro grupo cuida disso)